# CNN Ford Hyperparemter Tuning

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Normalization
%matplotlib inline
import seaborn as sns
from keras.models import Sequential 
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, InputLayer, SimpleRNN
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

from sklearn import (
    linear_model, metrics, neural_network, pipeline, model_selection
)
from sklearn.impute import SimpleImputer

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import TimeSeriesSplit

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [3]:
Ford = pd.read_csv("Ford_Cleaned_Date.csv")
Ford.date = pd.to_datetime(Ford.date)
Ford = Ford.set_index("date")
Ford = Ford.iloc[14:, :] # to remove first 14 days that include NaNs due to some calculations
Ford = Ford.drop(["Nas_total", 'Stock_total', 'Dow_total'],axis=1) # to remove duplicated columns
pd.DataFrame(Ford.isna().sum()).T

,Ford,F-150,Ford Bronco_x,Ford Mustang_x,Ford Stock,Open,High,Low,Close,Volume,...,Dow_MAvg_s_Move,Dow_EMA_Move,Dow_Disparity_Move,Dow_Disparity_s_Move,Dow_RSI_Move,target_1,target_2,target_3,target_4,target_5
0,0,0,0,0,0,6,6,6,6,6,...,0,0,0,0,0,0,0,0,0,0


In [4]:
Ford = Ford.dropna()
Ford = Ford[~(Ford.isin([np.inf, -np.inf]).any(axis=1))] 

In [5]:
Ford = Ford.drop(['target_1', 'target_2', 'target_4', 'target_5'], axis=1)
target_3 = Ford["target_3"]

#splitting into training sets 
column_indices = {name: i for i, name in enumerate(Ford.columns)}

n = len(Ford)
X_train = Ford[0:int(n*0.7)]
X_val = Ford[int(n*0.7):int(n*0.9)]
X_test = Ford[int(n*0.9):]

y_train = target_3[0:int(n*0.7)]
y_val = target_3[int(n*0.7):int(n*0.9)]
y_test = target_3[int(n*0.9):]

In [6]:
Mscaler = MinMaxScaler() # keeps binarys at zero and 1 :)

X_train = pd.DataFrame(Mscaler.fit_transform(X_train), columns = Ford.columns)
X_val = pd.DataFrame(Mscaler.fit_transform(X_val), columns = Ford.columns)

In [7]:
def df_to_X_y2(df, target, window_size=5):
  df_as_np = df.to_numpy() # converts to matrix of numpy arrays
  X = []
  y = []
  for i in range(len(df_as_np)-window_size): # length of data frame - window_size so it does't take empty values at the end, 
    # does force you to loose the last 5 values, could fix with padding
    row = [r for r in df_as_np[i:i+window_size]] # grabs row i and all rows above within the window size length
    X.append(row) # creates 3 dimentional array, (# obseravtions, # rows in window, # features)
    label = target[i+window_size] # pulls the target variable after the window, target varible needs to be column zero in this 
    y.append(label) # returns (N,) martix of targets i+window_length time periods away
  return np.array(X), np.array(y)

In [8]:
def kbest_creator(k):
    """
    returns list of k best features and the number of efficient principle compents to use with said k features
    """
    # apply SelectKBest class to extract top 40 best features
    bestfeatures = SelectKBest(score_func=f_regression, k=k)
    best_fit = bestfeatures.fit(X_train, y_train)
    best_scores = pd.DataFrame(best_fit.scores_)
    best_columns = pd.DataFrame(Ford.columns)
    
    # concatenate the dataframes for better visualization
    features_score = pd.concat([best_columns, best_scores], axis=1)
    features_score.columns = ['Features', 'Score']  # naming the dataframe columns
    feats = list(features_score.nlargest(k, 'Score')['Features'])

    pca = PCA().fit(X_train[feats])
    pca_scores = np.cumsum(pca.explained_variance_ratio_) * 100
    res = next(x for x, val in enumerate(pca_scores) if val > 85)
    res = res +1
    res
    
    return feats, res

## Model Testing

In [9]:
early_stopping_monitor = EarlyStopping(patience=4)

In [10]:
# Model1

feats, comp = kbest_creator(5)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [11]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model1_1 = Sequential()
model1_1.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model1_1.add(Flatten())
model1_1.add(Dense(25, activation='relu')) 
model1_1.add(Dense(1, activation='sigmoid'))

model1_1.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

2022-05-13 15:56:04.064649: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 15ms/step - loss: 0.6977 - binary_accuracy: 0.4501 - val_loss: 0.6930 - val_binary_accuracy: 0.5182
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6890 - binary_accuracy: 0.5418 - val_loss: 0.6926 - val_binary_accuracy: 0.5182
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6865 - binary_accuracy: 0.5397 - val_loss: 0.6947 - val_binary_accuracy: 0.4526
Epoch 4/30
16/16 [==============================] - 0s 3ms/step - loss: 0.6823 - binary_accuracy: 0.5845 - val_loss: 0.6946 - val_binary_accuracy: 0.4526
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6810 - binary_accuracy: 0.5621 - val_loss: 0.6961 - val_binary_accuracy: 0.5036
Epoch 6/30
16/16 [==============================] - 0s 3ms/step - loss: 0.6773 - binary_accuracy: 0.5703 - val_loss: 0.6952 - val_binary_accuracy: 0.4818


In [12]:
model1_2 = Sequential()
model1_2.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model1_2.add(Conv1D(filters=15, kernel_size=2, activation='relu'))
# better without pooling layer
model1_2.add(Flatten())
model1_2.add(Dense(30, activation='relu')) 
model1_2.add(Dense(30, activation='relu')) 
model1_2.add(Dense(1, activation='sigmoid'))

model1_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model1_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 27ms/step - loss: 0.6947 - binary_accuracy: 0.4623 - val_loss: 0.6945 - val_binary_accuracy: 0.5182
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6922 - binary_accuracy: 0.5234 - val_loss: 0.6941 - val_binary_accuracy: 0.5182
Epoch 3/30
16/16 [==============================] - 0s 10ms/step - loss: 0.6906 - binary_accuracy: 0.5214 - val_loss: 0.6953 - val_binary_accuracy: 0.5182
Epoch 4/30
16/16 [==============================] - 0s 13ms/step - loss: 0.6891 - binary_accuracy: 0.5336 - val_loss: 0.6949 - val_binary_accuracy: 0.5036
Epoch 5/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6874 - binary_accuracy: 0.5316 - val_loss: 0.6967 - val_binary_accuracy: 0.5182
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6861 - binary_accuracy: 0.5377 - val_loss: 0.6993 - val_binary_accuracy: 0.5036


In [13]:
# Model2

feats, comp = kbest_creator(10)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [14]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model2_1 = Sequential()
model2_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_1.add(Flatten())
model2_1.add(Dense(25, activation='relu')) 
model2_1.add(Dense(1, activation='sigmoid'))

model2_1.compile(optimizer='adagrad', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 19ms/step - loss: 0.6980 - binary_accuracy: 0.4745 - val_loss: 0.7005 - val_binary_accuracy: 0.4818
Epoch 2/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6978 - binary_accuracy: 0.4745 - val_loss: 0.7003 - val_binary_accuracy: 0.4964
Epoch 3/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6976 - binary_accuracy: 0.4745 - val_loss: 0.7002 - val_binary_accuracy: 0.4964
Epoch 4/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6975 - binary_accuracy: 0.4745 - val_loss: 0.7000 - val_binary_accuracy: 0.5036
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6973 - binary_accuracy: 0.4766 - val_loss: 0.6999 - val_binary_accuracy: 0.5036
Epoch 6/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6972 - binary_accuracy: 0.4766 - val_loss: 0.6998 - val_binary_accuracy: 0.5036
Epoch 7/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6970 - binar

In [15]:
model2_2 = Sequential()
model2_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model2_2.add(Flatten())
model2_2.add(Dense(25, activation='relu')) 
model2_2.add(Dense(1, activation='sigmoid'))

model2_2.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 2s 14ms/step - loss: 0.7000 - binary_accuracy: 0.4949 - val_loss: 0.6906 - val_binary_accuracy: 0.5036
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6909 - binary_accuracy: 0.5112 - val_loss: 0.6903 - val_binary_accuracy: 0.4818
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6873 - binary_accuracy: 0.5275 - val_loss: 0.6918 - val_binary_accuracy: 0.4964
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6836 - binary_accuracy: 0.5642 - val_loss: 0.6944 - val_binary_accuracy: 0.5255
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6816 - binary_accuracy: 0.5356 - val_loss: 0.6951 - val_binary_accuracy: 0.5255
Epoch 6/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6785 - binary_accuracy: 0.5682 - val_loss: 0.6973 - val_binary_accuracy: 0.5182


In [16]:
model2_3 = Sequential()
model2_3.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model2_3.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model2_3.add(MaxPooling1D(pool_size=2)) 
# better without pooling layer
model2_3.add(Flatten())
model2_3.add(Dense(30, activation='relu')) 
model2_3.add(Dense(1, activation='sigmoid'))

model2_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_3.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 14ms/step - loss: 0.6920 - binary_accuracy: 0.5214 - val_loss: 0.6953 - val_binary_accuracy: 0.5182
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6877 - binary_accuracy: 0.5295 - val_loss: 0.6959 - val_binary_accuracy: 0.4891
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6864 - binary_accuracy: 0.5479 - val_loss: 0.6976 - val_binary_accuracy: 0.5255
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6848 - binary_accuracy: 0.5519 - val_loss: 0.6985 - val_binary_accuracy: 0.5182
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6844 - binary_accuracy: 0.5560 - val_loss: 0.7006 - val_binary_accuracy: 0.5182


In [17]:
model2_4 = Sequential()
model2_4.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model2_4.add(Conv1D(filters=32, kernel_size=2, activation='relu')) 
model2_4.add(MaxPooling1D(pool_size=2)) 
model2_4.add(Conv1D(filters=32, kernel_size=1, activation='relu')) 
model2_4.add(Flatten())
model2_4.add(Flatten())
model2_4.add(Dense(25, activation='relu')) 
model2_4.add(Dense(1, activation='sigmoid'))

model2_4.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model2_4.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 2s 22ms/step - loss: 0.6947 - binary_accuracy: 0.5173 - val_loss: 0.6936 - val_binary_accuracy: 0.5182
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6905 - binary_accuracy: 0.5336 - val_loss: 0.6939 - val_binary_accuracy: 0.5182
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6886 - binary_accuracy: 0.5377 - val_loss: 0.6952 - val_binary_accuracy: 0.4891
Epoch 4/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6869 - binary_accuracy: 0.5397 - val_loss: 0.6974 - val_binary_accuracy: 0.4818
Epoch 5/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6842 - binary_accuracy: 0.5397 - val_loss: 0.7022 - val_binary_accuracy: 0.4599


In [18]:
# Model3

feats, comp = kbest_creator(25)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [19]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model3_1 = Sequential()
model3_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model3_1.add(Flatten())
model3_1.add(Dense(25, activation='relu')) 
model3_1.add(Dense(1, activation='sigmoid'))

model3_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 36ms/step - loss: 0.6963 - binary_accuracy: 0.5071 - val_loss: 0.6958 - val_binary_accuracy: 0.5182
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6873 - binary_accuracy: 0.5377 - val_loss: 0.6950 - val_binary_accuracy: 0.5036
Epoch 3/30
16/16 [==============================] - 0s 11ms/step - loss: 0.6829 - binary_accuracy: 0.5764 - val_loss: 0.6955 - val_binary_accuracy: 0.5109
Epoch 4/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6782 - binary_accuracy: 0.5825 - val_loss: 0.6976 - val_binary_accuracy: 0.4964
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6746 - binary_accuracy: 0.5927 - val_loss: 0.6977 - val_binary_accuracy: 0.5328
Epoch 6/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6710 - binary_accuracy: 0.6334 - val_loss: 0.7007 - val_binary_accuracy: 0.4818


In [20]:
model3_2 = Sequential()
model3_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model3_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model3_2.add(MaxPooling1D(pool_size=2)) 
# better without pooling layer
model3_2.add(Flatten())
model3_2.add(Dense(30, activation='relu')) 
model3_2.add(Dense(1, activation='sigmoid'))

model3_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model3_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 14ms/step - loss: 0.6985 - binary_accuracy: 0.4603 - val_loss: 0.6939 - val_binary_accuracy: 0.4745
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6902 - binary_accuracy: 0.5621 - val_loss: 0.6938 - val_binary_accuracy: 0.4891
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6861 - binary_accuracy: 0.6008 - val_loss: 0.6938 - val_binary_accuracy: 0.5109
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6827 - binary_accuracy: 0.6029 - val_loss: 0.6940 - val_binary_accuracy: 0.4745
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6780 - binary_accuracy: 0.6375 - val_loss: 0.6949 - val_binary_accuracy: 0.4891
Epoch 6/30
16/16 [==============================] - 0s 13ms/step - loss: 0.6738 - binary_accuracy: 0.6293 - val_loss: 0.6945 - val_binary_accuracy: 0.5620


In [21]:
# Model4

feats, comp = kbest_creator(50)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [22]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model4_1 = Sequential()
model4_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model4_1.add(Flatten())
model4_1.add(Dense(25, activation='relu')) 
model4_1.add(Dense(1, activation='sigmoid'))

model4_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 12ms/step - loss: 0.7213 - binary_accuracy: 0.4623 - val_loss: 0.6950 - val_binary_accuracy: 0.5182
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6949 - binary_accuracy: 0.5214 - val_loss: 0.6907 - val_binary_accuracy: 0.5328
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6823 - binary_accuracy: 0.5866 - val_loss: 0.6895 - val_binary_accuracy: 0.5401
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6724 - binary_accuracy: 0.6130 - val_loss: 0.6901 - val_binary_accuracy: 0.5474
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6626 - binary_accuracy: 0.6395 - val_loss: 0.6912 - val_binary_accuracy: 0.5474
Epoch 6/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6547 - binary_accuracy: 0.6538 - val_loss: 0.6914 - val_binary_accuracy: 0.5328
Epoch 7/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6468 - binar

In [23]:
model4_2 = Sequential()
model4_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model4_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model4_2.add(MaxPooling1D(pool_size=2)) 
# better without pooling layer
model4_2.add(Flatten())
model4_2.add(Dense(30, activation='relu')) 
model4_2.add(Dense(1, activation='sigmoid'))

model4_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model4_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 13ms/step - loss: 0.7035 - binary_accuracy: 0.4705 - val_loss: 0.7010 - val_binary_accuracy: 0.4526
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6879 - binary_accuracy: 0.5214 - val_loss: 0.6999 - val_binary_accuracy: 0.4818
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6825 - binary_accuracy: 0.5784 - val_loss: 0.6993 - val_binary_accuracy: 0.4818
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6752 - binary_accuracy: 0.6212 - val_loss: 0.6985 - val_binary_accuracy: 0.4964
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6693 - binary_accuracy: 0.6334 - val_loss: 0.6992 - val_binary_accuracy: 0.5036
Epoch 6/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6631 - binary_accuracy: 0.6456 - val_loss: 0.6999 - val_binary_accuracy: 0.5036
Epoch 7/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6

In [24]:
# Model5

feats, comp = kbest_creator(100)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 5 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [25]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model5_1 = Sequential()
model5_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model5_1.add(Flatten())
model5_1.add(Dense(25, activation='relu')) 
model5_1.add(Dense(1, activation='sigmoid'))

model5_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model5_1.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 14ms/step - loss: 0.7091 - binary_accuracy: 0.5499 - val_loss: 0.7003 - val_binary_accuracy: 0.5328
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6811 - binary_accuracy: 0.5703 - val_loss: 0.6986 - val_binary_accuracy: 0.5401
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6666 - binary_accuracy: 0.6232 - val_loss: 0.6995 - val_binary_accuracy: 0.5401
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6524 - binary_accuracy: 0.6436 - val_loss: 0.7001 - val_binary_accuracy: 0.5255
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6420 - binary_accuracy: 0.6823 - val_loss: 0.7045 - val_binary_accuracy: 0.4891
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6295 - binary_accuracy: 0.6762 - val_loss: 0.7067 - val_binary_accuracy: 0.4964


In [26]:
model5_2 = Sequential()
model5_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model5_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model5_2.add(MaxPooling1D(pool_size=2)) 
# better without pooling layer
model5_2.add(Flatten())
model5_2.add(Dense(30, activation='relu')) 
model5_2.add(Dense(1, activation='sigmoid'))

model5_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model5_2.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 14ms/step - loss: 0.7050 - binary_accuracy: 0.5071 - val_loss: 0.7002 - val_binary_accuracy: 0.5328
Epoch 2/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6843 - binary_accuracy: 0.5418 - val_loss: 0.7016 - val_binary_accuracy: 0.4818
Epoch 3/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6761 - binary_accuracy: 0.5825 - val_loss: 0.7034 - val_binary_accuracy: 0.4672
Epoch 4/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6681 - binary_accuracy: 0.6151 - val_loss: 0.7049 - val_binary_accuracy: 0.5036
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6608 - binary_accuracy: 0.6090 - val_loss: 0.7097 - val_binary_accuracy: 0.5182


In [27]:
model5_3 = Sequential()
model5_3.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model5_3.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model5_3.add(MaxPooling1D(pool_size=2)) 
model5_3.add(Conv1D(filters=16, kernel_size=1, activation='relu'))
# better without pooling layer
model5_3.add(Flatten())
model5_3.add(Dense(30, activation='relu')) 
model5_3.add(Dense(1, activation='sigmoid'))

model5_3.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model5_3.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 15ms/step - loss: 0.6971 - binary_accuracy: 0.5275 - val_loss: 0.6904 - val_binary_accuracy: 0.5693
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6837 - binary_accuracy: 0.5397 - val_loss: 0.6920 - val_binary_accuracy: 0.5474
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6756 - binary_accuracy: 0.5723 - val_loss: 0.6928 - val_binary_accuracy: 0.5109
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6667 - binary_accuracy: 0.5927 - val_loss: 0.6918 - val_binary_accuracy: 0.5255
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6578 - binary_accuracy: 0.6090 - val_loss: 0.6922 - val_binary_accuracy: 0.5474


In [28]:
#Model6 Starting to Adjust Window Size
feats, comp = kbest_creator(40)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 3 periods
X_train_1, train_3w = df_to_X_y2(X_train_1,y_train,3)
X_val_1, val_3w = df_to_X_y2(X_val_1, y_val, 3)
X_test_1, test_3w = df_to_X_y2(X_test_1,y_test, 3) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [29]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model6_1 = Sequential()
model6_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model6_1.add(Flatten())
model6_1.add(Dense(25, activation='relu')) 
model6_1.add(Dense(1, activation='sigmoid'))

model6_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model6_1.fit(X_train_1, train_3w,epochs=30,  validation_data=(X_val_1, val_3w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 12ms/step - loss: 0.7063 - binary_accuracy: 0.4888 - val_loss: 0.6956 - val_binary_accuracy: 0.5683
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6920 - binary_accuracy: 0.5355 - val_loss: 0.6938 - val_binary_accuracy: 0.5612
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6859 - binary_accuracy: 0.5436 - val_loss: 0.6935 - val_binary_accuracy: 0.5396
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6805 - binary_accuracy: 0.5619 - val_loss: 0.6937 - val_binary_accuracy: 0.5108
Epoch 5/30
16/16 [==============================] - 0s 3ms/step - loss: 0.6761 - binary_accuracy: 0.5842 - val_loss: 0.6954 - val_binary_accuracy: 0.5180
Epoch 6/30
16/16 [==============================] - 0s 3ms/step - loss: 0.6724 - binary_accuracy: 0.6146 - val_loss: 0.6963 - val_binary_accuracy: 0.5180
Epoch 7/30
16/16 [==============================] - 0s 3ms/step - loss: 0.6683 - binar

In [30]:
model6_2 = Sequential()
model6_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model6_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model6_2.add(MaxPooling1D(pool_size=1)) #has to be one, I'm guessing becuase of decrease in window size
# better without pooling layer
model6_2.add(Flatten())
model6_2.add(Dense(30, activation='relu')) 
model6_2.add(Dense(1, activation='sigmoid'))

model6_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model6_2.fit(X_train_1, train_3w,epochs=30,  validation_data=(X_val_1, val_3w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 14ms/step - loss: 0.6981 - binary_accuracy: 0.4767 - val_loss: 0.6951 - val_binary_accuracy: 0.4748
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6892 - binary_accuracy: 0.5619 - val_loss: 0.6950 - val_binary_accuracy: 0.5108
Epoch 3/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6845 - binary_accuracy: 0.5963 - val_loss: 0.6944 - val_binary_accuracy: 0.5036
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6799 - binary_accuracy: 0.6146 - val_loss: 0.6953 - val_binary_accuracy: 0.5108
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6750 - binary_accuracy: 0.6450 - val_loss: 0.6951 - val_binary_accuracy: 0.4820
Epoch 6/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6696 - binary_accuracy: 0.6673 - val_loss: 0.6953 - val_binary_accuracy: 0.5108
Epoch 7/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6

In [31]:
#Model7 Starting to Adjust Window Size
feats, comp = kbest_creator(40)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 7 periods
X_train_1, train_7w = df_to_X_y2(X_train_1,y_train,7)
X_val_1, val_7w = df_to_X_y2(X_val_1, y_val, 7)
X_test_1, test_7w = df_to_X_y2(X_test_1,y_test, 7) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [32]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model7_1 = Sequential()
model7_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model7_1.add(Flatten())
model7_1.add(Dense(25, activation='relu')) 
model7_1.add(Dense(1, activation='sigmoid'))

model7_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model7_1.fit(X_train_1, train_7w,epochs=30,  validation_data=(X_val_1, val_7w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 16ms/step - loss: 0.7014 - binary_accuracy: 0.4847 - val_loss: 0.7001 - val_binary_accuracy: 0.5111
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6826 - binary_accuracy: 0.5685 - val_loss: 0.6992 - val_binary_accuracy: 0.5111
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6709 - binary_accuracy: 0.5971 - val_loss: 0.7004 - val_binary_accuracy: 0.5185
Epoch 4/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6632 - binary_accuracy: 0.6176 - val_loss: 0.6964 - val_binary_accuracy: 0.4741
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6524 - binary_accuracy: 0.6585 - val_loss: 0.7001 - val_binary_accuracy: 0.4667
Epoch 6/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6435 - binary_accuracy: 0.6503 - val_loss: 0.6963 - val_binary_accuracy: 0.4593
Epoch 7/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6329 - binar

In [33]:
model7_2 = Sequential()
model7_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model7_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model7_2.add(MaxPooling1D(pool_size=1)) #has to be one, I'm guessing becuase of decrease in window size
# better without pooling layer
model7_2.add(Flatten())
model7_2.add(Dense(30, activation='relu')) 
model7_2.add(Dense(1, activation='sigmoid'))

model7_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model7_2.fit(X_train_1, train_7w,epochs=30,  validation_data=(X_val_1, val_7w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 14ms/step - loss: 0.6981 - binary_accuracy: 0.4806 - val_loss: 0.6957 - val_binary_accuracy: 0.5185
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6804 - binary_accuracy: 0.5542 - val_loss: 0.6956 - val_binary_accuracy: 0.5407
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6698 - binary_accuracy: 0.6217 - val_loss: 0.6940 - val_binary_accuracy: 0.4889
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6585 - binary_accuracy: 0.6524 - val_loss: 0.6931 - val_binary_accuracy: 0.5037
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6468 - binary_accuracy: 0.6810 - val_loss: 0.6952 - val_binary_accuracy: 0.4667
Epoch 6/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6331 - binary_accuracy: 0.7055 - val_loss: 0.6948 - val_binary_accuracy: 0.4741
Epoch 7/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6

In [34]:
#Model8 Starting to Adjust Window Size
feats, comp = kbest_creator(40)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 14 periods
X_train_1, train_14w = df_to_X_y2(X_train_1,y_train,14)
X_val_1, val_14w = df_to_X_y2(X_val_1, y_val, 14)
X_test_1, test_14w = df_to_X_y2(X_test_1,y_test, 14) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [35]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model8_1 = Sequential()
model8_1.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 

model8_1.add(Flatten())
model8_1.add(Dense(25, activation='relu')) 
model8_1.add(Dense(1, activation='sigmoid'))

model8_1.compile(optimizer='RMSProp', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model8_1.fit(X_train_1, train_14w,epochs=30,  validation_data=(X_val_1, val_14w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 13ms/step - loss: 0.7032 - binary_accuracy: 0.5207 - val_loss: 0.6985 - val_binary_accuracy: 0.4531
Epoch 2/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6699 - binary_accuracy: 0.6162 - val_loss: 0.7009 - val_binary_accuracy: 0.5391
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6506 - binary_accuracy: 0.6639 - val_loss: 0.7121 - val_binary_accuracy: 0.5156
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6372 - binary_accuracy: 0.6722 - val_loss: 0.7127 - val_binary_accuracy: 0.4531
Epoch 5/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6208 - binary_accuracy: 0.7324 - val_loss: 0.7009 - val_binary_accuracy: 0.5469


In [36]:
model8_2 = Sequential()
model8_2.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_steps,n_features))) 
model8_2.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
model8_2.add(MaxPooling1D(pool_size=1)) #has to be one, I'm guessing becuase of decrease in window size
# better without pooling layer
model8_2.add(Flatten())
model8_2.add(Dense(30, activation='relu')) 
model8_2.add(Dense(1, activation='sigmoid'))

model8_2.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model8_2.fit(X_train_1, train_14w,epochs=30,  validation_data=(X_val_1, val_14w), callbacks = [early_stopping_monitor])

Epoch 1/30
16/16 [==============================] - 1s 14ms/step - loss: 0.7008 - binary_accuracy: 0.4979 - val_loss: 0.7131 - val_binary_accuracy: 0.5078
Epoch 2/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6714 - binary_accuracy: 0.6266 - val_loss: 0.7125 - val_binary_accuracy: 0.4062
Epoch 3/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6570 - binary_accuracy: 0.6805 - val_loss: 0.7161 - val_binary_accuracy: 0.4141
Epoch 4/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6423 - binary_accuracy: 0.6784 - val_loss: 0.7209 - val_binary_accuracy: 0.4219
Epoch 5/30
16/16 [==============================] - 0s 5ms/step - loss: 0.6204 - binary_accuracy: 0.6909 - val_loss: 0.7233 - val_binary_accuracy: 0.3750
Epoch 6/30
16/16 [==============================] - 0s 4ms/step - loss: 0.6069 - binary_accuracy: 0.7220 - val_loss: 0.7257 - val_binary_accuracy: 0.4062


In [37]:
#Model9 Early RNN
feats, comp = kbest_creator(40)

sklearn_pca = PCA(n_components=comp)
X_train_1 = pd.DataFrame(sklearn_pca.fit_transform(X_train[feats]))
X_val_1 = pd.DataFrame(sklearn_pca.transform(X_val[feats]))
X_test_1 = pd.DataFrame(sklearn_pca.transform(X_test[feats]))

# converting to window format, in this case 14 periods
X_train_1, train_5w = df_to_X_y2(X_train_1,y_train,5)
X_val_1, val_5w = df_to_X_y2(X_val_1, y_val, 5)
X_test_1, test_5w = df_to_X_y2(X_test_1,y_test, 5) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


In [38]:
n_steps = X_train_1.shape[1]
n_features = X_train_1.shape[2]

model_9 = Sequential()
model_9.add(SimpleRNN(8,activation="relu", return_sequences=False, input_shape=(n_steps,n_features)))
model_9.add(Dense(30, activation='relu')) 
model_9.add(Dense(1, activation='sigmoid'))

model_9.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=["BinaryAccuracy"])

model_9.fit(X_train_1, train_5w,epochs=30,  validation_data=(X_val_1, val_5w), callbacks = [early_stopping_monitor])

Epoch 1/30


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


16/16 [==============================] - 1s 19ms/step - loss: 0.7014 - binary_accuracy: 0.4847 - val_loss: 0.6984 - val_binary_accuracy: 0.4672
Epoch 2/30
16/16 [==============================] - 0s 6ms/step - loss: 0.6963 - binary_accuracy: 0.5153 - val_loss: 0.6961 - val_binary_accuracy: 0.4964
Epoch 3/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6938 - binary_accuracy: 0.5295 - val_loss: 0.6943 - val_binary_accuracy: 0.5109
Epoch 4/30
16/16 [==============================] - 0s 8ms/step - loss: 0.6915 - binary_accuracy: 0.5418 - val_loss: 0.6925 - val_binary_accuracy: 0.5036
Epoch 5/30
16/16 [==============================] - 0s 12ms/step - loss: 0.6894 - binary_accuracy: 0.5438 - val_loss: 0.6917 - val_binary_accuracy: 0.4964
Epoch 6/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6875 - binary_accuracy: 0.5418 - val_loss: 0.6902 - val_binary_accuracy: 0.4891
Epoch 7/30
16/16 [==============================] - 0s 7ms/step - loss: 0.6857 - bina